# Exploring Instructor for Structured Output

Instructor makes it easy to get structured data like JSON from LLMs like GPT-3.5, GPT-4, GPT-4-Vision, and open-source models including Mistral/Mixtral, Ollama, and llama-cpp-python.

[Link to GitHub](https://python.useinstructor.com)

## Structured outputs with Ollama

In [1]:
!pip install instructor

  Attempting uninstall: jiter
    Found existing installation: jiter 0.7.1
    Uninstalling jiter-0.7.1:
      Successfully uninstalled jiter-0.7.1


In [1]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
from pprint import pprint
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

import instructor

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [38]:
llm = instructor.from_openai(OpenAI(
        base_url="http://ollama:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
    temperature=0.1
)

In [29]:
llm = instructor.from_openai(OpenAI(), mode=instructor.Mode.JSON)

In [4]:
llm

In [39]:
class Person(BaseModel):
    name: str = Field(description="Person's name")
    span: str = Field(description="Years active in sports")
    facts: List[str] = Field(description="A list of 5 interesting facts about the person")


In [55]:
# messages = [SystemMessage("You are a helpful assistant with profound knowledge in history. You are helping your human friends by answering their questions on important people of the past."),
#            HumanMessage("Can you tell me about 4 important people who lived in the 16th century.")]

In [49]:
messages_alt = [
    {"role": "system", "content": "You are a helpful assistant with profound knowledge in world sport. You are helping your human friends by answering their questions on famous people in sports."},
    {"role": "user", "content": "Can you compare and contrast between Dimitar Berbatov and Hristo Stoichkov"}
]

In [50]:
resp = llm.chat.completions.create(
    model="llama3.1:8b-instruct-fp16",
    messages=messages_alt,
    response_model=List[Person])

In [51]:
for i in resp:
    print(i.model_dump_json(indent=2))

{
  "name": "Dimitar Berbatov",
  "span": "1996-2017",
  "facts": [
    "He is a Bulgarian former professional footballer who played as a striker.",
    "Berbatov has won numerous titles, including the Premier League Golden Boot in 2009 and the UEFA Cup with Tottenham Hotspur in 2008.",
    "He was named the Bulgarian Footballer of the Year five times between 2004 and 2010.",
    "Berbatov made his international debut for Bulgaria in 1999 and earned over 80 caps, scoring 48 goals.",
    "He played for clubs such as Bayer Leverkusen, Tottenham Hotspur, Manchester United, Fulham, and Monaco during his career."
  ]
}
{
  "name": "Hristo Stoichkov",
  "span": "1982-2003",
  "facts": [
    "He is a Bulgarian former professional footballer who played as an attacking midfielder or forward.",
    "Stoichkov won the Golden Boot at the 1994 FIFA World Cup, scoring six goals in the tournament.",
    "He was named the European Footballer of the Year in 1994 and the FIFA World Player of the Year in

In [27]:
resp[1].name

'Isaac Newton'

### Defining additional Ollama models

So far we are only dealing with publicly available models, but it would be worth while to try and build one which is fine tuned.

In [70]:
from ollama import Client

cl = Client(host="ollama")


In [71]:
cl.pull(model="llama3.1:8b-instruct-fp16")

{'status': 'success'}